In [ ]:
# ===============================
# 0. Librairies
# ===============================
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
# ===============================
# 1. Generate complex synthetic time series
# ===============================
def create_complex_time_series(n_samples=2000, seq_len=50, n_features=3, seed=42):
    np.random.seed(seed)
    t = np.arange(n_samples + seq_len)
    
    series = []
    for f in range(n_features):
        # Different frequency sine/cosine
        freq = np.random.uniform(0.01, 0.1)
        amp = np.random.uniform(0.5, 2.0)
        wave = amp * np.sin(2 * np.pi * freq * t) + amp/2 * np.cos(2 * np.pi * freq*0.5 * t)
        
        # Add trend
        trend = t * np.random.uniform(0.0005, 0.002)
        
        # Add seasonal effect
        season = 0.5 * np.sin(2 * np.pi * t / np.random.randint(30, 100))
        
        # Add noise
        noise = np.random.normal(0, 0.2, len(t))
        
        # Add occasional spikes
        spikes = np.zeros(len(t))
        spike_idx = np.random.choice(len(t), size=int(0.01*len(t)), replace=False)
        spikes[spike_idx] = np.random.uniform(1, 3, len(spike_idx))
        
        series.append(wave + trend + season + noise + spikes)
    
    series = np.stack(series, axis=-1)
    
    # Build sequences
    X, y = [], []
    for i in range(n_samples):
        X.append(series[i:i+seq_len])
        y.append(series[i+seq_len, 0])  # predict first feature as example
    
    X = np.array(X)  # (n_samples, seq_len, n_features)
    y = np.array(y)  # (n_samples,)
    return X, y, series

SEQ_LEN = 50
N_FEATURES = 3
X, y, true_series = create_complex_time_series(n_samples=5000, seq_len=SEQ_LEN, n_features=N_FEATURES)

PRED_HORIZON = 20  # predict 20 steps ahead

def create_multi_step_dataset(series, seq_len=50, pred_horizon=20):
    X, Y = [], []
    N = len(series) - seq_len - pred_horizon
    for i in range(N):
        X.append(series[i:i+seq_len])
        Y.append(series[i+seq_len:i+seq_len+pred_horizon, 0])  # first feature
    return np.array(X), np.array(Y)

X_multi, y_multi = create_multi_step_dataset(np.concatenate([X[:, :, 0], X[:, :, 1], X[:, :, 2]], axis=1).reshape(-1, N_FEATURES), seq_len=SEQ_LEN, pred_horizon=PRED_HORIZON)

train_size = int(0.8 * len(X_multi))
X_train, X_val = X_multi[:train_size], X_multi[train_size:]
y_train, y_val = y_multi[:train_size], y_multi[train_size:]

In [ ]:
# ===============================
# 2. Plotting the time series
# ===============================
plt.figure(figsize=(14, 5))
plt.plot(true_series[:, 0], label="True underlying target series")
plt.title("True Continuous Time Series")
plt.legend()
plt.show()

In [ ]:
# ===============================
# 3. LSTM 
# ===============================
input_layer = layers.Input(shape=(SEQ_LEN, N_FEATURES))
x = layers.LSTM(64, return_sequences=True)(input_layer)
x = layers.MultiHeadAttention(num_heads=2, key_dim=32)(x, x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, activation='relu')(x)
output_layer = layers.Dense(PRED_HORIZON)(x)  # output multiple steps at once

lstm_model = models.Model(inputs=input_layer, outputs=output_layer)
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()

lstm_history = lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

In [ ]:
# ===============================
# 4. Plot only last-step predictions for each sequence
# ===============================
def plot_last_step_predictions(model, X, y_true, n_sequences=50):
    """
    Plots the last predicted value of each sequence vs the true last value
    
    X: (N, seq_len, n_features)
    y_true: (N, PRED_HORIZON) or (N,)
    n_sequences: number of sequences to plot
    """
    y_pred = model.predict(X[:n_sequences])
    
    # If multi-step, take only the last step
    if y_pred.ndim > 1:
        y_pred_last = y_pred[:, -1]
    else:
        y_pred_last = y_pred
    
    if y_true.ndim > 1:
        y_true_last = y_true[:n_sequences, -1]
    else:
        y_true_last = y_true[:n_sequences]
    
    plt.figure(figsize=(14, 5))
    plt.plot(y_true_last, label='True Last Value', color='blue')
    plt.plot(y_pred_last, label='Predicted Last Value', color='orange')
    plt.xlabel('Sequence Index')
    plt.ylabel('Value')
    plt.title('LSTM Last-Step Prediction for Each Sequence')
    plt.legend()
    plt.show()

# Example usage
plot_last_step_predictions(lstm_model, X_val, y_val, n_sequences=200)

In [ ]:
# ===============================
# 5. Direct Multi-step Forecasting
# ===============================
def direct_multi_step_forecast(model, X_input):
    """
    X_input: (1, seq_len, n_features) initial sequence
    Returns: array of predicted steps (PRED_HORIZON,)
    """
    y_pred = model.predict(X_input[np.newaxis, :, :])
    return y_pred[0]  # shape (PRED_HORIZON,)

# Choose last validation sequence as starting point
X_start = X_val[-1]
future_lstm = direct_multi_step_forecast(lstm_model, X_start)

plt.figure(figsize=(14, 5))
plt.plot(np.arange(len(y_val[-200:])), y_val[-200:, 0], label='Recent True Values', color='blue')
plt.plot(np.arange(len(y_val[-200:]), len(y_val[-200:]) + PRED_HORIZON),
         future_lstm, label='LSTM Multi-step Forecast', color='orange')
plt.xlabel('Time step')
plt.ylabel('Value')
plt.title('Multi-step Future Forecasting (Direct)')
plt.legend()
plt.show()

In [ ]:
# ===============================
# 6. Recursive Multi-step Forecasting
# ===============================
def recursive_multi_forecast(model, X_init, n_future=100):
    """
    Predict n_future steps recursively from initial sequence X_init
    X_init: (seq_len, n_features)
    Returns: array of length n_future
    """
    seq = X_init.copy()
    predictions = []

    for _ in range(n_future):
        # Predict next PRED_HORIZON steps
        pred = model.predict(seq[np.newaxis, :, :])[0]  # shape: (PRED_HORIZON,)
        next_step = pred[0]  # take only first step for recursion
        predictions.append(next_step)

        # Update sequence: remove first step, append predicted step
        new_input = np.zeros(seq.shape[1])
        new_input[0] = next_step  # first feature predicted
        seq = np.vstack([seq[1:], new_input])

    return np.array(predictions)

def past_predictions_from_multistep(model, X):
    """
    Returns past predictions aligned with y by taking ONLY the last step
    of each horizon prediction.
    """
    pred = model.predict(X)              # shape (N, pred_horizon)
    return pred[:, 0]                   # last step of each horizon

In [ ]:
# ===============================
# 7. Plot Predictions + Future Forecasts
# ===============================
def plot_predictions_and_forecast(model, X_val, y_val, n_past=200, n_future=200, label="Model"):
    """
    Works with MULTI-STEP LSTM (model outputs pred_horizon steps)
    """
    # --- past predictions (use last horizon step)
    y_pred_past = past_predictions_from_multistep(model, X_val[-n_past:])
    
    # --- recursive future forecast
    X_start = X_val[-1]    # last window
    future_pred = recursive_multi_forecast(model, X_start, n_future)

    # --- Plot ---
    plt.figure(figsize=(14, 6))

    # True past
    plt.plot(range(n_past), y_val[-n_past:, 0], label="True (Past)", color="blue")

    # Predicted past
    plt.plot(range(n_past), y_pred_past, label=f"Predicted (Past) - {label}", color="orange")

    # Forecast future
    plt.plot(range(n_past, n_past + n_future), future_pred, label=f"Future Forecast - {label}", color="green")

    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.title(f"{label} — Past Predictions + {n_future} Step Recursive Forecast")
    plt.legend()
    plt.show()

# Plot for LSTM
plot_predictions_and_forecast(lstm_model, X_val, y_val, n_past=200, n_future=100, label="LSTM")

In [ ]:
# ===============================
# 8. Plot Predictions + Long Future Forecasts
# ===============================
plot_predictions_and_forecast(lstm_model, X_val, y_val, n_past=1000, n_future=400, label="LSTM")